In [11]:
import os
import cv2

In [30]:
def crop_using_edges(image):
    # Check if the image is grayscale (1 channel) or color (3 channels)
    if len(image.shape) == 2:  # Grayscale image
        gray = image  # No conversion needed
    elif len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError("Invalid image format: The image must be either grayscale or color (BGR).")

    #Apply Egde detection
    edges = cv2.Canny(gray, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Get the bounding box of the largest contour
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Crop the image to the bounding box
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image  # Return the original image if no contours are found

In [13]:
def crop_fixed_border(image, top=10, bottom=10, left=10, right=10):
    cropped_image = image[top:image.shape[0]-bottom, left:image.shape[1]-right]
    return cropped_image

In [28]:
def crop_using_contours(image):
    # Check if the image is grayscale (1 channel) or color (3 channels)
    if len(image.shape) == 2:  # Grayscale image
        gray = image  # No conversion needed
    elif len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError("Invalid image format: The image must be either grayscale or color (BGR).")

    #Apply thresholding
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image


In [15]:
def crop_based_on_intensity(image, threshold=240):
    # Convert to grayscale if necessary
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) > 2 else image

    # Threshold the background
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image


In [ ]:
def crop_using_morphology(image):
    # Check if the image is grayscale (1 channel) or color (3 channels)
    if len(image.shape) == 2:  # Grayscale image
        gray = image  # No conversion needed
    elif len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError("Invalid image format: The image must be either grayscale or color (BGR).")
    
    # Apply thresholding
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV)
    
    # Morphological closing to connect ridge lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Find contours and crop based on the largest contour
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image


In [17]:
def remove_border(image_path, crop_pixels=2):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply a binary threshold to separate the fingerprint from the background
    _, thresh = cv2.threshold(image, 10, 255, cv2.THRESH_BINARY)
    
    # Find contours of the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # If no contours are found, return the original image
    if not contours:
        return image
    
    # Find the bounding rectangle of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Adjust the coordinates to crop the specified number of pixels from the border
    x = max(x + crop_pixels, 0)  # Ensure x does not go negative
    y = max(y + crop_pixels, 0)  # Ensure y does not go negative
    w = max(w - 2 * crop_pixels, 0)  # Reduce width
    h = max(h - 2 * crop_pixels, 0)  # Reduce height
    
    # Crop the image to the adjusted bounding rectangle
    cropped_image = image[y:y+h, x:x+w]
    
    return cropped_image


In [33]:
image_path = "SOCOFing\\Real\\597__M_Right_little_finger.BMP"

image = remove_border(image_path, 2)

cropped_image1 = crop_using_morphology(image)
cropped_image2 = crop_based_on_intensity(image)
cropped_image3 = crop_fixed_border(image)
cropped_image4 = crop_using_contours(image)
cropped_image5 = crop_using_edges(image)

# Resize window
cv2.namedWindow("Morp", cv2.WINDOW_NORMAL)
cv2.namedWindow("Intensity", cv2.WINDOW_NORMAL)
cv2.namedWindow("Border", cv2.WINDOW_NORMAL)
cv2.namedWindow("Contours", cv2.WINDOW_NORMAL)
cv2.namedWindow("Edges", cv2.WINDOW_NORMAL)

# Display the cropped image
cv2.imshow("Morp", cropped_image1)
cv2.imshow("Intensity", cropped_image2)
cv2.imshow("Border", cropped_image3)
cv2.imshow("Contours", cropped_image4)
cv2.imshow("Edges", cropped_image5)

cv2.waitKey(0)
cv2.destroyAllWindows()
